In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
import time
import copy
from sklearn.metrics import precision_score, f1_score, confusion_matrix
import seaborn as sns
import os
import sys
from datetime import datetime

Select device and using logger for report

In [8]:
# Base directory for storing all experiment results
base_output_dir = "experiment_results"
os.makedirs(base_output_dir, exist_ok=True)

def create_experiment_dir(base_dir=base_output_dir):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    exp_dir = os.path.join(base_dir, f"experiment_{timestamp}")
    os.makedirs(exp_dir, exist_ok=True)  # Ensure directory creation is safe
    return exp_dir

class Logger:
    def __init__(self, filename):
        self.terminal = sys.stdout
        self.log = open(filename, "a")  # Append mode to keep previous logs

    def write(self, message):
        self.terminal.write(message)
        self.log.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {message}")  # Add timestamp
        self.flush()

    def flush(self):
        # Ensure both streams are flushed immediately 
        self.terminal.flush()
        self.log.flush()

    def __del__(self):
        self.log.close()  # Ensure the log file is closed when the object is deleted

# Set up device - use GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


data pre-processing

In [9]:
# Data augmentation and normalization for training
transform_train = transforms.Compose([
   transforms.RandomCrop(32, padding=4),
   transforms.RandomHorizontalFlip(),
   transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
   transforms.RandomRotation(15),  # Random rotation
   transforms.ToTensor(),
   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Only normalization for testing/validation
transform_test = transforms.Compose([
   transforms.ToTensor(),
   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Set data path variable
data_path = "./data"
full_trainset = datasets.CIFAR10(root=data_path, train=True, download=True, transform=transform_train)

# Dynamically calculate training and validation set sizes
total_size = len(full_trainset)
train_size = int(0.9 * total_size)
val_size = total_size - train_size
trainset, valset = random_split(full_trainset, [train_size, val_size])

# Load test set
testset = datasets.CIFAR10(root=data_path, train=False, download=True, transform=transform_test)

# Create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=100, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# CIFAR10 classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
   """
   Train and validate the model over specified epochs
   Returns trained model and training history
   """
   since = time.time()
   best_model_wts = copy.deepcopy(model.state_dict())
   best_acc = 0.0
   
   # Initialize history lists for tracking metrics
   train_loss_history = []
   train_acc_history = []
   val_loss_history = []
   val_acc_history = [] 

   for epoch in range(num_epochs):
       print(f'Epoch {epoch}/{num_epochs - 1}')
       print('-' * 10)
       # Each epoch has a training and validation phase
       for phase in ['train', 'val']:
           if phase == 'train':
               model.train() 
               dataloader = trainloader
           else:
               model.eval()  
               dataloader = valloader

           running_loss = 0.0
           running_corrects = 0
           # Iterate over data batches
           for inputs, labels in dataloader:
               inputs = inputs.to(device)
               labels = labels.to(device)

               # Zero the parameter gradients
               optimizer.zero_grad()

               # Forward pass
               # Track history only in train phase
               with torch.set_grad_enabled(phase == 'train'):
                   outputs = model(inputs)
                   _, preds = torch.max(outputs, 1)
                   loss = criterion(outputs, labels)

                   # Backward pass + optimize only in training phase
                   if phase == 'train':
                       loss.backward()
                       optimizer.step()
               # Statistics
               running_loss += loss.item() * inputs.size(0)
               running_corrects += torch.sum(preds == labels.data)

           # Learning rate scheduling
           if phase == 'train':
               scheduler.step()

           # Calculate epoch metrics
           epoch_loss = running_loss / len(dataloader.dataset)
           epoch_acc = running_corrects.double() / len(dataloader.dataset)

           print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

           # Record history
           if phase == 'train':
               train_loss_history.append(epoch_loss)
               train_acc_history.append(epoch_acc.cpu().numpy())
           else:
               val_loss_history.append(epoch_loss)
               val_acc_history.append(epoch_acc.cpu().numpy())

           # Save best model
           if phase == 'val' and epoch_acc > best_acc:
               best_acc = epoch_acc
               best_model_wts = copy.deepcopy(model.state_dict())

   # Print training time and load best model
   time_elapsed = time.time() - since
   print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
   print(f'Best val Acc: {best_acc:4f}')
   model.load_state_dict(best_model_wts)
   
   return model, train_loss_history, train_acc_history, val_loss_history, val_acc_history

def evaluate_model(model, dataloader):
   """
   Evaluate model performance and calculate metrics
   Returns accuracy, precision, F1 score and predictions
   """
   model.eval()
   all_preds = []
   all_labels = []
   
   # Predict without gradient computation
   with torch.no_grad():
       for inputs, labels in dataloader:
           inputs = inputs.to(device)
           labels = labels.to(device)
           outputs = model(inputs)
           _, preds = torch.max(outputs, 1)
           all_preds.extend(preds.cpu().numpy())
           all_labels.extend(labels.cpu().numpy())
   
   # Calculate metrics
   all_preds = np.array(all_preds)
   all_labels = np.array(all_labels)
   accuracy = (all_preds == all_labels).mean()
   precision = precision_score(all_labels, all_preds, average='macro')
   f1 = f1_score(all_labels, all_preds, average='macro')
   
   return accuracy, precision, f1, all_preds, all_labels



In [11]:
def plot_training_history(train_loss, train_acc, val_loss, val_acc, save_path):
   """
   Plot training/validation loss and accuracy curves
   Save the plot to specified path
   """
   plt.figure(figsize=(12, 4))
   
   # Plot loss
   plt.subplot(1, 2, 1)
   plt.plot(train_loss, label='Train Loss')
   plt.plot(val_loss, label='Validation Loss')
   plt.xlabel('Epoch')
   plt.ylabel('Loss')
   plt.legend()
   plt.title('Training and Validation Loss')

   # Plot accuracy
   plt.subplot(1, 2, 2)
   plt.plot(train_acc, label='Train Accuracy') 
   plt.plot(val_acc, label='Validation Accuracy')
   plt.xlabel('Epoch')
   plt.ylabel('Accuracy') 
   plt.legend()
   plt.title('Training and Validation Accuracy')
   
   plt.tight_layout()
   plt.savefig(save_path)
   plt.close()

def plot_confusion_matrix(y_true, y_pred, classes, save_path):
   """
   Create and save confusion matrix heatmap
   """
   cm = confusion_matrix(y_true, y_pred)
   plt.figure(figsize=(10, 8))
   sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
               xticklabels=classes, yticklabels=classes)
   plt.title('Confusion Matrix')
   plt.xlabel('Predicted')
   plt.ylabel('True')
   plt.tight_layout()
   plt.savefig(save_path)
   plt.close()

def print_results(model_name, best_val_acc, test_acc, test_precision, test_f1, train_time):
   """
   Print evaluation metrics in a formatted way
   """
   print(f"Results for {model_name}:")
   print(f"Best Validation Accuracy: {best_val_acc:.4f}")
   print(f"Test Accuracy: {test_acc:.4f}")
   print(f"Test Precision: {test_precision:.4f}")
   print(f"Test F1 Score: {test_f1:.4f}")
   print(f"Training Time: {train_time:.2f} seconds")
   print("-" * 40)

In [12]:
def train_resnet18(lr=0.01, momentum=0.9, weight_decay=5e-4, exp_dir=None):
    
   model = models.resnet18(pretrained=True)
   num_ftrs = model.fc.in_features
   model.fc = nn.Linear(num_ftrs, 10) 
   model = model.to(device)

   # 定义损失函数和优化器
   criterion = nn.CrossEntropyLoss()
   optimizer = optim.SGD(model.parameters(), 
                        lr=lr, 
                        momentum=momentum, 
                        weight_decay=weight_decay)
   
   # 学习率调度器：每 7 个 epoch 将学习率减少 0.1 倍
   scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

   # 训练模型并记录时间
   start_time = time.time()
   model, train_loss, train_acc, val_loss, val_acc = train_model(
       model, criterion, optimizer, scheduler, num_epochs=25)
   end_time = time.time()

   # 绘制训练曲线
   plot_training_history(train_loss, train_acc, val_loss, val_acc, 
                        os.path.join(exp_dir, 'resnet18_training_history.png'))
   
   # 在测试集上进行评估并打印评估指标
   test_acc, test_precision, test_f1, y_pred, y_true = evaluate_model(model, testloader)
   print_results("ResNet18", max(val_acc), test_acc, test_precision, test_f1, 
                end_time - start_time)
   
   # 绘制混淆矩阵
   plot_confusion_matrix(y_true, y_pred, classes, 
                        os.path.join(exp_dir, 'resnet18_confusion_matrix.png'))


In [13]:
def train_vgg16(lr=0.01, momentum=0.9, weight_decay=5e-4, exp_dir=None):

   model = models.vgg16(pretrained=True)
   num_ftrs = model.classifier[6].in_features 
   model.classifier[6] = nn.Linear(num_ftrs, 10) 
   model = model.to(device)

   # 定义损失函数和优化器，与 ResNet18 使用相同的超参数
   criterion = nn.CrossEntropyLoss()
   optimizer = optim.SGD(model.parameters(), 
                        lr=lr,
                        momentum=momentum, 
                        weight_decay=weight_decay)
   
   # 学习率调度器：每 7 个 epoch 将学习率减少 0.1 倍
   scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

   # 训练模型并记录时间
   start_time = time.time()
   model, train_loss, train_acc, val_loss, val_acc = train_model(
       model, criterion, optimizer, scheduler, num_epochs=25)
   end_time = time.time()

   # 绘制训练曲线
   plot_training_history(train_loss, train_acc, val_loss, val_acc,
                        os.path.join(exp_dir, 'vgg16_training_history.png'))
   
   # 在测试集上进行评估并打印评估指标
   test_acc, test_precision, test_f1, y_pred, y_true = evaluate_model(model, testloader)
   print_results("VGG16", max(val_acc), test_acc, test_precision, test_f1,
                end_time - start_time)
   
   # 绘制混淆矩阵
   plot_confusion_matrix(y_true, y_pred, classes,
                        os.path.join(exp_dir, 'vgg16_confusion_matrix.png'))


In [14]:
def train_mobilenetv2(lr=0.01, momentum=0.9, weight_decay=5e-4, exp_dir=None):
   
   model = models.mobilenet_v2(pretrained=True)
   num_ftrs = model.classifier[1].in_features 
   model.classifier[1] = nn.Linear(num_ftrs, 10) 
   model = model.to(device)

   # 定义损失函数和优化器，超参数与其他模型相同
   criterion = nn.CrossEntropyLoss()
   optimizer = optim.SGD(model.parameters(), 
                        lr=lr,
                        momentum=momentum, 
                        weight_decay=weight_decay)
   
   # 学习率调度器：每 7 个 epoch 将学习率减少 0.1 倍
   scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

   # 训练模型并记录时间
   start_time = time.time()
   model, train_loss, train_acc, val_loss, val_acc = train_model(
       model, criterion, optimizer, scheduler, num_epochs=25)
   end_time = time.time()

   # 绘制训练曲线
   plot_training_history(train_loss, train_acc, val_loss, val_acc,
                        os.path.join(exp_dir, 'mobilenetv2_training_history.png'))
   
   # 在测试集上进行评估并打印评估指标
   test_acc, test_precision, test_f1, y_pred, y_true = evaluate_model(model, testloader)
   print_results("MobileNetV2", max(val_acc), test_acc, test_precision, test_f1,
                end_time - start_time)
   
   # 绘制混淆矩阵
   plot_confusion_matrix(y_true, y_pred, classes,
                        os.path.join(exp_dir, 'mobilenetv2_confusion_matrix.png'))

In [15]:
def run_experiments():

   # 创建带有时间戳的新实验目录
   exp_dir = create_experiment_dir()
   
   # 设置日志记录，将控制台输出保存到文件
   sys.stdout = Logger(os.path.join(exp_dir, 'experiment_log.txt'))

   # MobileNetV2 实验，包含两种超参数设置
   print("\n开始 MobileNetV2 实验...")
   # 默认设置
   train_mobilenetv2(lr=0.01, momentum=0.9, weight_decay=5e-4, exp_dir=exp_dir)
   # 修改后的设置，使用更低的学习率和更高的动量
   train_mobilenetv2(lr=0.001, momentum=0.95, weight_decay=1e-4, exp_dir=exp_dir)

   # ResNet18 实验，使用相同的超参数变化
   print("开始 ResNet18 实验...")
   train_resnet18(lr=0.01, momentum=0.9, weight_decay=5e-4, exp_dir=exp_dir)
   train_resnet18(lr=0.001, momentum=0.95, weight_decay=1e-4, exp_dir=exp_dir)

   # VGG16 实验，使用相同的超参数变化
   print("\n开始 VGG16 实验...")
   train_vgg16(lr=0.01, momentum=0.9, weight_decay=5e-4, exp_dir=exp_dir)
   train_vgg16(lr=0.001, momentum=0.95, weight_decay=1e-4, exp_dir=exp_dir)

   # 恢复标准输出
   sys.stdout = sys.__stdout__

In [16]:
if __name__ == "__main__":
    run_experiments()


开始 MobileNetV2 实验...


c:\Users\lenovo\.conda\envs\py38\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lenovo\.conda\envs\py38\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/24
----------
train Loss: 1.3747 Acc: 0.5166
val Loss: 1.0399 Acc: 0.6384
Epoch 1/24
----------
train Loss: 0.9711 Acc: 0.6647
val Loss: 0.9298 Acc: 0.6778
Epoch 2/24
----------
train Loss: 0.8752 Acc: 0.6949
val Loss: 0.8370 Acc: 0.7048
Epoch 3/24
----------
train Loss: 0.8175 Acc: 0.7170
val Loss: 0.7957 Acc: 0.7254
Epoch 4/24
----------
train Loss: 0.7744 Acc: 0.7304
val Loss: 0.8101 Acc: 0.7128
Epoch 5/24
----------
train Loss: 0.7490 Acc: 0.7397
val Loss: 0.7503 Acc: 0.7378
Epoch 6/24
----------
train Loss: 0.7201 Acc: 0.7490
val Loss: 0.7030 Acc: 0.7646
Epoch 7/24
----------
train Loss: 0.6323 Acc: 0.7806
val Loss: 0.6134 Acc: 0.7824
Epoch 8/24
----------
train Loss: 0.5945 Acc: 0.7935
val Loss: 0.5905 Acc: 0.7898
Epoch 9/24
----------
train Loss: 0.5822 Acc: 0.7960
val Loss: 0.5824 Acc: 0.7948
Epoch 10/24
----------
train Loss: 0.5711 Acc: 0.8000
val Loss: 0.5824 Acc: 0.7984
Epoch 11/24
----------
train Loss: 0.5640 Acc: 0.8035
val Loss: 0.5856 Acc: 0.7952
Epoch 12/24
--

c:\Users\lenovo\.conda\envs\py38\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lenovo\.conda\envs\py38\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


train Loss: 1.4234 Acc: 0.4881
val Loss: 1.0775 Acc: 0.6176
Epoch 1/24
----------
train Loss: 1.0315 Acc: 0.6354
val Loss: 0.9102 Acc: 0.6818
Epoch 2/24
----------
train Loss: 0.9048 Acc: 0.6814
val Loss: 0.8476 Acc: 0.7036
Epoch 3/24
----------
train Loss: 0.8449 Acc: 0.7033
val Loss: 0.8207 Acc: 0.7176
Epoch 4/24
----------
train Loss: 0.8082 Acc: 0.7163
val Loss: 0.7572 Acc: 0.7344
Epoch 5/24
----------
train Loss: 0.7610 Acc: 0.7327
val Loss: 0.7596 Acc: 0.7312
Epoch 6/24
----------
train Loss: 0.7418 Acc: 0.7399
val Loss: 0.7338 Acc: 0.7406
Epoch 7/24
----------
train Loss: 0.6982 Acc: 0.7542
val Loss: 0.6830 Acc: 0.7628
Epoch 8/24
----------
train Loss: 0.6821 Acc: 0.7597
val Loss: 0.6889 Acc: 0.7568
Epoch 9/24
----------
train Loss: 0.6680 Acc: 0.7647
val Loss: 0.6760 Acc: 0.7622
Epoch 10/24
----------
train Loss: 0.6587 Acc: 0.7680
val Loss: 0.6765 Acc: 0.7622
Epoch 11/24
----------
train Loss: 0.6613 Acc: 0.7680
val Loss: 0.6715 Acc: 0.7676
Epoch 12/24
----------
train Loss: 0

...

Epoch 0/24
----------
train Loss: 1.4226 Acc: 0.4946
val Loss: 1.0894 Acc: 0.6122
Epoch 1/24
----------
train Loss: 1.0274 Acc: 0.6398
val Loss: 0.9283 Acc: 0.6802
Epoch 2/24
----------
train Loss: 0.9076 Acc: 0.6810
val Loss: 0.8443 Acc: 0.7114
Epoch 3/24
----------
train Loss: 0.8302 Acc: 0.7093
val Loss: 0.8153 Acc: 0.7164
Epoch 4/24
----------
train Loss: 0.7829 Acc: 0.7266
val Loss: 0.7649 Acc: 0.7370
Epoch 5/24
----------
train Loss: 0.7401 Acc: 0.7424
val Loss: 0.7456 Acc: 0.7364
Epoch 6/24
----------
train Loss: 0.7134 Acc: 0.7510
val Loss: 0.7165 Acc: 0.7508
Epoch 7/24
----------
train Loss: 0.6666 Acc: 0.7662
val Loss: 0.6827 Acc: 0.7624
Epoch 8/24
----------
train Loss: 0.6509 Acc: 0.7718
val Loss: 0.6888 Acc: 0.7600
Epoch 9/24
----------
train Loss: 0.6447 Acc: 0.7735
val Loss: 0.6818 Acc: 0.7648
Epoch 10/24
----------
train Loss: 0.6393 Acc: 0.7768
val Loss: 0.6678 Acc: 0.7680
Epoch 11/24
----------
train Loss: 0.6364 Acc: 0.7766
val Loss: 0.6793 Acc: 0.7674
Epoch 12/24
--

c:\Users\lenovo\.conda\envs\py38\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lenovo\.conda\envs\py38\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/24
----------
train Loss: 1.1255 Acc: 0.6109
val Loss: 0.7749 Acc: 0.7382
Epoch 1/24
----------
train Loss: 0.7290 Acc: 0.7535
val Loss: 0.6475 Acc: 0.7844
Epoch 2/24
----------
train Loss: 0.6079 Acc: 0.7948
val Loss: 0.5772 Acc: 0.8012
Epoch 3/24
----------
train Loss: 0.5546 Acc: 0.8141
val Loss: 0.5355 Acc: 0.8284
Epoch 4/24
----------
train Loss: 0.4989 Acc: 0.8318
val Loss: 0.5140 Acc: 0.8260
Epoch 5/24
----------
train Loss: 0.4672 Acc: 0.8435
val Loss: 0.4881 Acc: 0.8350
Epoch 6/24
----------
train Loss: 0.4323 Acc: 0.8550
val Loss: 0.4655 Acc: 0.8450
Epoch 7/24
----------
train Loss: 0.3209 Acc: 0.8922
val Loss: 0.4059 Acc: 0.8586
Epoch 8/24
----------
train Loss: 0.2892 Acc: 0.9017
val Loss: 0.3878 Acc: 0.8712
Epoch 9/24
----------
train Loss: 0.2765 Acc: 0.9048
val Loss: 0.3889 Acc: 0.8734
Epoch 10/24
----------
train Loss: 0.2650 Acc: 0.9088
val Loss: 0.3693 Acc: 0.8796
Epoch 11/24
----------
train Loss: 0.2583 Acc: 0.9116
val Loss: 0.3769 Acc: 0.8740
Epoch 12/24
--

c:\Users\lenovo\.conda\envs\py38\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lenovo\.conda\envs\py38\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/24
----------
train Loss: 1.1930 Acc: 0.5781
val Loss: 0.8557 Acc: 0.6986
Epoch 1/24
----------
train Loss: 0.7825 Acc: 0.7324
val Loss: 0.6938 Acc: 0.7600
Epoch 2/24
----------
train Loss: 0.6642 Acc: 0.7751
val Loss: 0.6333 Acc: 0.7856
Epoch 3/24
----------
train Loss: 0.5989 Acc: 0.7962
val Loss: 0.5855 Acc: 0.7972
Epoch 4/24
----------
train Loss: 0.5503 Acc: 0.8138
val Loss: 0.5300 Acc: 0.8170
Epoch 5/24
----------
train Loss: 0.5218 Acc: 0.8212
val Loss: 0.5283 Acc: 0.8214
Epoch 6/24
----------
train Loss: 0.4893 Acc: 0.8328
val Loss: 0.5159 Acc: 0.8196
Epoch 7/24
----------
train Loss: 0.4209 Acc: 0.8561
val Loss: 0.4591 Acc: 0.8440
Epoch 8/24
----------
train Loss: 0.4074 Acc: 0.8585
val Loss: 0.4770 Acc: 0.8362
Epoch 9/24
----------
train Loss: 0.4044 Acc: 0.8611
val Loss: 0.4584 Acc: 0.8444
Epoch 10/24
----------
train Loss: 0.4019 Acc: 0.8619
val Loss: 0.4583 Acc: 0.8476
Epoch 11/24
----------
train Loss: 0.3887 Acc: 0.8630
val Loss: 0.4566 Acc: 0.8460
Epoch 12/24
--